In [0]:
# LSTM for sequence classification
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Convolution1D, Flatten, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import TensorBoard
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


Using TensorFlow backend.


In [0]:
from google.colab import files 
uploaded = files.upload() 

Saving threshold4_balanced_training_set.csv to threshold4_balanced_training_set.csv


In [0]:
df = pd.read_csv('threshold4_balanced_training_set.csv', usecols=['CONTENT','sentiment'])
df.head(10)
df['label']=df['sentiment']
df['label'].value_counts()

neutral     1269
positive    1269
negative    1269
Name: label, dtype: int64

In [0]:
for i, item in enumerate (df['label']):
  if item=='positive':
    df['label'][i]=2
  if item=='neutral':
    df['label'][i]=1
  if item=='negative':
    df['label'][i]=0
df['label'].value_counts()

2    1269
1    1269
0    1269
Name: label, dtype: int64

In [0]:
from google.colab import files 
uploaded = files.upload() 

Saving test.csv to test.csv


In [0]:
tdf = pd.read_csv('test.csv')
len(tdf)
x = df['CONTENT']
y = df['label']
x_test=tdf['CONTENT']
y_test=tdf['label']

In [0]:
import sklearn.model_selection
from sklearn.model_selection import train_test_split
SEED = 2000
x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=0.2, random_state=SEED)
print (y_train)

3390    0
1297    2
3320    0
3466    0
1987    2
       ..
1659    2
1590    2
1230    1
840     1
2998    0
Name: label, Length: 3045, dtype: object


In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=42225)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)
sequences_val=tokenizer.texts_to_sequences(x_validation)

In [0]:

# Pad the sequence to the same length
max_review_length = 15000
x_train = pad_sequences(sequences, maxlen=max_review_length)
x_validation = pad_sequences(sequences_val, maxlen=max_review_length)



In [0]:
# Using embedding from Keras
# from keras.utils import to_categorical
# y_binary = to_categorical(y_int)
embedding_vecor_length = 300
top_words = 42225
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))

# Convolutional model (3x conv, flatten, 2x dense)
model.add(Convolution1D(64, 3, padding='same'))
model.add(Convolution1D(32, 3, padding='same'))
model.add(Convolution1D(16, 3, padding='same'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(180,activation='softmax'))
model.add(Dropout(0.2))
model.add(Dense(3,activation='softmax'))

# Log to tensorboard
tensorBoardCallback = TensorBoard(log_dir='./logs', write_graph=True)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train.astype('int'), epochs=3, callbacks=[tensorBoardCallback], batch_size=64)

# Evaluation on the test set
scores = model.evaluate(x_validation, y_validation.astype('int'), verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/3
3045/3045 [==============================] - 636s 209ms/step - loss: 1.1036 - acc: 0.3268

Epoch 2/3
3045/3045 [==============================] - 624s 205ms/step - loss: 1.1005 - acc: 0.3314
Epoch 3/3
3045/3045 [==============================] - 639s 210ms/step - loss: 1.0999 - acc: 0.3284
Accuracy: 34.78%


In [0]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
embedding_size=32
vocabulary_size=42225
max_words=15000
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(3, activation='softmax'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 15000, 32)         1351200   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 303       
Total params: 1,404,703
Trainable params: 1,404,703
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
model.compile(loss='sparse_categorical_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [0]:
batch_size = 64
num_epochs = 5

model.fit(x_train, y_train.astype('int'), batch_size=batch_size, epochs=num_epochs)

Epoch 1/5
3045/3045 [==============================] - 1056s 347ms/step - loss: 0.6401 - acc: 0.7307
Epoch 2/5
3045/3045 [==============================] - 1063s 349ms/step - loss: 0.5092 - acc: 0.7928
Epoch 3/5
3045/3045 [==============================] - 1060s 348ms/step - loss: 0.4386 - acc: 0.8167
Epoch 4/5
3045/3045 [==============================] - 1060s 348ms/step - loss: 0.3898 - acc: 0.8368
Epoch 5/5
3045/3045 [==============================] - 1064s 349ms/step - loss: 0.3672 - acc: 0.8516


In [0]:
scores = model.evaluate(x_validation, y_validation.astype('int'), verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.4356955380577428
